PROJECTE DE PIM:  INPAINTING

Autors: Oriol Riu & Carlos Navas

In [32]:
# Importem les llibreries necesaries
from skimage import data, draw, color, data, restoration, transform, util, filters 
from scipy import misc, fftpack, signal, ndimage
import numpy as np
from numpy.linalg import inv
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageEnhance
import matplotlib.animation as animation
from scipy.misc import imresize
from scipy import ndimage as nd
from random import randint
import scipy

# Funcio que normalitzara una imatge
def normalitzar(imatge):
    return np.real((imatge - np.min(imatge)) * 256 / (np.max(imatge) - np.min(imatge)))

scale = 5
rgb =  plt.imread('casa3.jpg')
# Redimensionem la imatge que hem llegit i la convertim en una imatge en escala de grissos
rgb = imresize(rgb, ( rgb.shape[0]/scale,rgb.shape[1]/scale,3),interp='bilinear')
img_LAB = color.rgb2lab(rgb.astype('float'))
img_gris = color.rgb2gray(rgb)

# Normalitzem la imatge original y la imatge en escala de gris
rgb = normalitzar(rgb)
img_gris = normalitzar(img_gris)

In [33]:
class get_mouse_click(): 
    #Mouse interaction interface for radial distortion removal.
    def __init__(self, img):
      height, width = img.shape[:2]
      self.figure = plt.imshow(img, extent=(0, width, height, 0))
      plt.gray()
      plt.title('select the object to remove')
      plt.xlabel('Select sets of  points with left mouse button,\n'
                 'click right button to close the polygon.')
      plt.connect('button_press_event', self.button_press)
      plt.connect('motion_notify_event', self.mouse_move)

      self.img = np.atleast_3d(img)
      self.points = []
      self.centre = np.array([(width - 1)/2., (height - 1)/2.])

      self.height = height
      self.width = width

      self.make_cursorline()
      self.figure.axes.set_autoscale_on(False)

      plt.show()
      plt.close()

    def make_cursorline(self):
        self.cursorline, = plt.plot([0],[0],'r:+', linewidth=2,markersize=15,markeredgecolor='b')

    def button_press(self,event):
        # Register mouse clicks.
        if (event.button == 1 and event.xdata and event.ydata):
            self.points.append((event.xdata,event.ydata))
            plt.gca().lines.append(self.cursorline)
            plt.gca()
            self.make_cursorline()
        if (event.button != 1):
            self.points.append((self.points[0][0],self.points[0][1]))
            plt.close()
            return self.points

    def mouse_move(self,event):
        # Handle cursor drawing.
        pts_last_set=len(self.points)
        pts = np.zeros((pts_last_set+1,2))
        if pts_last_set > 0:
            # Line follows up to 3 clicked points:
            pts[:pts_last_set] = self.points[-pts_last_set:]
            # The last point of the line follows the mouse cursor
        pts[pts_last_set:] = [event.xdata,event.ydata]
        self.cursorline.set_data(pts[:,0], pts[:,1])
        plt.draw()
        
# Funcio que retorna la mascara escollida         
def compute_mask(width, height, polygon):
    img = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
    mask = np.array(img)
    return mask

# Funcio que retornara l'imatge amb els forats amb valor -100
def convine_mask(zona_delete, gris):
    # Recorrem tots els pixels de l'imatge
    for i in xrange(len(gris)):
        for j in xrange(len(gris[0])):
            #si tenim 1 a la macara tenim forat, per tant, li fiquem un -100
            if zona_delete[i][j]==1:
                gris[i][j]=-100
    return gris

# Funcio amb la cual recorrerem l'imatge y trobarem el millor pixel per a pintar
def reccorrer_img(finestra,finestra_validar,img_gris,t,r,vent):
    dif = 19999999990.
    list_valors = []
    count = 0
    for i in xrange(len(img_gris)-vent):
        for j in xrange(len(img_gris[0])-vent):
            # Trobem quins seran els millors pixels
            valor = millors_pix(finestra,finestra_validar,img_gris,i,j)
            if valor < dif:                           # Si el valor es el minim i el valor diferent de -100
                if img_gris_convine[i][j] != -100:
                    valors = i, j
                    dif = valor
                    # Fiquem a una llista de valors, que seran els valors aceptats.
                    list_valors.append(valors)
    # Retornem la llista amb els valors aceptats
    return list_valors[len(list_valors)-1]

# Funcio amb la cual trobarem els millors pixels, seran els mes semblants als que volem trobar
def millors_pix(finestra,finestra_validar,img_gris,t,r):
    ssd=0.
    total_ones_mask = finestra_validar.sum()
    #aqui fariem el calcul de la formula explicada pel professor, la formula es la del SSD
    for i in xrange(len(finestra_validar)):
        for j in xrange(len(finestra_validar[0])):
            ssd += (finestra_validar[i][j]*((img_gris[t+i,r+j] - finestra[i][j])**2))
    if total_ones_mask==0:
        print "panic"
    # Retornem el resultat que ens dona la formula SSD
    return (1./total_ones_mask)*ssd

FINESTRES:

Segons la zona on volem fer inpainting tindrem que escollir una finestra o una altre aixo es perque segons el tamany de la finestra escollira millor els pixels en aquesta zona en concret.

* Paret = 8.

* Cel = 6.

* Taulada = Con una ventana de 4 y de 2 lo hace bastante bien, aunque a veces hay que pasarlo 2 veces

* Herba = 8.

In [41]:
# Truquem a la funcio get_mouse_click per a seleccionar la part de l'imatge on volem fer l'inpainting
rdi = get_mouse_click(img_gris.astype('uint8'))
height, width = img_gris.shape

# Truquem a la funcio compute_mask per que ens retorni la zona on volem fer inpainting
zona_delete = compute_mask(width, height, rdi.points)

vent = 4      # Variable amb el tamany de la finestra
cont = 0
img_gris_convine = img_gris                     # La variable img_gris_convine contindra tota l'area seleccionada a -100
img_seam = normalitzar(color.rgb2gray(rgb))
seams_extraccio = []
seams_fi = []

# Truquem a la funcio convine_mask que retorna l'imatge amb els forats amb valor -100
img_search = convine_mask(zona_delete,img_gris)

# Recorrem tota la zona a esborrar
while zona_delete.sum()>=1:
    # Fem l'erosio de la zona que volem borrar
    zona_delete_erosio = ndimage.binary_erosion(zona_delete)

    # Busco el contorn mes extern de la part a substituir
    delete_ceva = zona_delete - zona_delete_erosio 
    # Actualitzo la mascara amb el que ja haurem substituit
    # el problema ara esta en que al fer els pixels de la dreta el millor resultat que troba es el negre
    finestra = np.zeros((vent,vent), dtype=np.int)
    # Truquem a la funcio convine_mask que retorna l'imatge amb els forats amb valor -100
    img_gris_convine = convine_mask(zona_delete, img_gris_convine)

    # Recorrem tota l'imatge delete_ceva pixel per pixel i si ens trobem un pixel a 1 significara que tenim que pintar
    for i in xrange(len(delete_ceva)):
        for j in xrange(len(delete_ceva[0])):
            if delete_ceva[i][j] == 1:
                delete_ceva[i][j] = 0
                finestra_validar = np.ones((vent,vent), dtype=np.int)
                finestra_validar_tmp = np.ones((vent,vent), dtype=np.int)
                # Agafem una finestra amb un tamany que li hem posat avanç
                finestra=img_gris_convine[i-vent/2:i+vent/2,j-vent/2:j+vent/2]  
                finestra_validar_tmp=zona_delete[i-vent/2:i+vent/2,j-vent/2:j+vent/2]
                finestra_validar=abs(finestra_validar_tmp-finestra_validar)
                # Recorrem l'imatge y retornem el conjunt de valors aceptats per a pintarlos a aquest punt
                # Aquests valors seran aquells pixels de l'imatge que segons la nostra finestra mes s'asemblen
                t, r = reccorrer_img(finestra, finestra_validar, img_gris_convine, i, j, vent)
                print img_gris_convine[t][r], img_gris_convine[i][j]
                img_seam[t][r] = 255
                seams_extraccio.append([plt.imshow(img_seam.astype('uint8'))]) 
                # El pixel que volem pintar sera el de l'imatge convine
                img_gris[i][j] = img_gris_convine[t][r]
                misc.imsave('imatges/teulada'+str(cont+1)+'.png', img_gris.astype("uint8"))
                cont = cont + 1
    # La zona a esborrar ara sera la zona d'erosio, per anant fent-ho pas a pas
    zona_delete = zona_delete_erosio

184.379379665 -100.0
196.36550396 -100.0
205.898159438 -100.0
215.344803835 -100.0
212.346671785 -100.0
213.586821272 -100.0
217.873684165 -100.0
196.36550396 -100.0
200.84459444 -100.0
188.725313068 -100.0
206.727465243 -100.0
217.873684165 -100.0
213.586821272 -100.0
214.477516244 -100.0
214.630326686 -100.0
214.221691247 -100.0
217.163402679 -100.0
184.379379665 -100.0
210.46745666 -100.0
215.344803835 -100.0
212.346671785 -100.0
207.770970657 -100.0
171.263738875 -100.0


In [42]:
# Mostrem l'animacio dels pixels que agafem per a pintar el forat 
fig = plt.figure(1)
ani = animation.ArtistAnimation(fig, seams_extraccio, interval=100, blit=True, repeat_delay=1000);

# Mostrem l'imatge corregida amb l'inpainting
plt.show();
plt.imshow(img_gris.astype('uint8'))
plt.title('Imatge corregida')
plt.show()